In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from lsst.ts.wep import Instrument
from lsst.ts.wep.utils import createZernikeBasis
import galsim

In [ ]:
# Create Zernike basis
inst = Instrument(pixelSize=0.1e-5)
u, v = inst.createPupilGrid()
zk = createZernikeBasis(u, v, jmin=1)
noll_indices = np.arange(1, 23)

# Mask outside annulus
r = np.sqrt(u**2 + v**2)
mask = (r >= 0.61) & (r <= 1)

# Apply mask
zk *= mask

In [ ]:
# Create title dictionary
titles = {
    1: "Piston",
    2: "Tilt",
    3: "Tilt",
    4: "Defocus",
    5: "Astigmatism",
    6: "Astigmatism",
    7: "Coma",
    8: "Coma",
    9: "Trefoil",
    10: "Trefoil",
    11: "Spherical",
    12: "2nd Astigmatism",
    13: "2nd Astigmatism",
    14: "Quadrafoil",
    15: "Quadrafoil",
    16: "2nd Coma",
    17: "2nd Coma",
    18: "2nd Trefoil",
    19: "2nd Trefoil",
    20: "Pentafoil",
    21: "Pentafoil",
    22: "2nd Spherical",
}

In [ ]:
# Create the figure
fig = plt.Figure(figsize=(7.5, 9), dpi=120)

# Use Josh's code to setup pyramid
jmin = min(noll_indices)
jmax = max(noll_indices)
jdict = {x: y for x, y in zip(noll_indices, range(len(noll_indices)))}
nmax, _ = galsim.zernike.noll_to_zern(jmax)
nmin, _ = galsim.zernike.noll_to_zern(jmin)

nrow = nmax - nmin + 1
ncol = nmax + 1
gridspec = GridSpec(nrow, ncol)

def shift(pos, amt):
    return [pos.x0 + amt, pos.y0, pos.width, pos.height]

def shiftAxes(axes, amt):
    for ax in axes:
        ax.set_position(shift(ax.get_position(), amt))

axes = {}
shiftLeft = []
shiftRight = []
for j in noll_indices:
    n, m = galsim.zernike.noll_to_zern(j)
    if n % 2 == 0:
        row, col = n - nmin, m // 2 + ncol // 2
    else:
        row, col = n - nmin, (m - 1) // 2 + ncol // 2
    subplotspec = gridspec.new_subplotspec((row, col))
    axes[j] = fig.add_subplot(subplotspec)
    axes[j].set_aspect("equal")
    if nmax % 2 == 0 and (nmax - n) % 2 == 1:
        shiftRight.append(axes[j])
    if nmax % 2 == 1 and (nmax - n) % 2 == 1:
        shiftLeft.append(axes[j])
        
# Loop through and plot polynomials
for j, ax in axes.items():
    if j not in noll_indices:
        continue
    ax.set_title(f"{j}\n{titles[j]}", fontsize=10)
    ax.imshow(zk[jdict[j]], origin="lower", cmap="bwr")

    # Make sure colors are symmetric about 0
    img = ax.get_images()[0]
    vmax = np.abs(img.get_clim()).max()
    img.set_clim(-vmax, +vmax)
    
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect("equal")
    ax.set_axis_off()

fig.tight_layout()

# Assume we always have Z4 and Z5?
amt = 0.5 * (axes[4].get_position().x0 - axes[5].get_position().x0)
shiftAxes(shiftLeft, -amt)
shiftAxes(shiftRight, amt)

fig.savefig("figures/pyramid.pdf")

In [ ]:
fig, axes = plt.subplots(3, 6, figsize=(7, 4), dpi=150)

def plot_zk(j, ax):
    # Plot image
    ax.imshow(zk[j - 1], origin="lower", cmap="bwr")

    # Make sure colors are symmetric about 0
    img = ax.get_images()[0]
    vmax = np.abs(img.get_clim()).max()
    img.set_clim(-vmax, +vmax)

    # Set title
    ax.text(
        0.5,
        1.05,
        f"{titles[j]}",
        ha="center",
        va="bottom",
        transform=ax.transAxes,
        fontsize=8,
    )
    
# nu = 0
plot_zk(4, axes[0, 0])
plot_zk(3, axes[0, 1])
plot_zk(5, axes[0, 2])
plot_zk(9, axes[0, 3])
plot_zk(15, axes[0, 4])
plot_zk(21, axes[0, 5])

# nu = 1
plot_zk(11, axes[1, 0])
plot_zk(7, axes[1, 1])
plot_zk(13, axes[1, 2])
plot_zk(19, axes[1, 3])

# nu = 2
plot_zk(22, axes[2, 0])
plot_zk(17, axes[2, 1])

for ax in axes.flatten():
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set(xticks=[], yticks=[], aspect="equal")

for i in range(len(axes)):
    axes[i, 0].set_ylabel(i, rotation=0, labelpad=18, fontsize=12, ha="center", va="center")

for i in range(len(axes[0])):
    axes[0, i].set_title(i, pad=25, fontsize=12)

fig.text(0.5, 1.04, "$m$", fontsize=16, ha="center", va="center")
fig.text(0.035, 0.5, "$\\nu$", fontsize=16, ha="center", va="center")

fig.savefig("figures/nu_table.pdf", bbox_inches="tight")